## **0. Installing essential libraries**

In [ ]:
!pip install PyMuPDF # for reading PDFs with Python
!pip install tqdm # for progress bar
!pip install yake # for keyword extraction
!pip install sentence-transformers # for embedding models
!pip install accelerate # for quantization model loading
!pip install bitsandbytes # for quantizing models (less storage space)
!pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference!pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.5 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.6.1-cp310-cp310-linux_x86_64.whl size=198447665 sha256=808523ff263d2fc1d3801147ad90d89dbfca67623a110c26592aef2064ea7b65
  Stored in directory: /root/.cache/pip/wheels/91/6a/38/f0faa036b4ac73a73247386f1ab1bb4cb4f6e72e6861a779f1
Successfully built flash-attn


##**1. Data Ingestion**

In [ ]:
import fitz
import os
from tqdm.auto import tqdm # for progress bar

In [ ]:
%%time
def extract_text_from_all_pdfs(directory: str)-> list[dict]:
    """
    Extracts text from all PDF files in a directory.
    """
    resumes = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            pdf_document = fitz.open(file_path)
            text = ""
            for page_num, page in tqdm(enumerate(pdf_document)):
                text += page.get_text()
            resumes.append({"filename": filename,
                              "page_count": page_num + 1,
                              "page_char_count": len(text),
                              "page_word_count": len(text.split(" ")),
                              "page_sentence_count_raw": len(text.split(". ")),
                              "page_token_count": len(text) / 4,  # 1 token
                              "text": text})
            pdf_document.close()
    return resumes

resumes = extract_text_from_all_pdfs('/content/')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

CPU times: user 157 ms, sys: 12.1 ms, total: 169 ms
Wall time: 201 ms


In [ ]:
%%time
def read_txt_file(file_path: str)-> str:
  """
  Reads a text file and returns its contents as a string.
  """
  with open(file_path, 'r') as file:
    text = file.read()
  return text

file_path = '/content/desc.txt'
description = read_txt_file(file_path)

CPU times: user 775 µs, sys: 52 µs, total: 827 µs
Wall time: 631 µs


In [ ]:
description

'Job Summary\n\nThe Digital Marketing Executive is responsible to generate enquiries for courses offered by IIAS and support the admission services by developing IIAS’ presence across digital platforms. You will plan, develop, implement, track and optimize our digital marketing campaigns across all digital channels. You will work with the marketing team, design and development team to launch campaigns on time and on budget.\n\nJob Description\n• Design and execute online campaigns that help build brand awareness and engagement.\n• Actively strategize and implement innovative social media campaigns.\n• Manage day-to-day operations including conceptualizing the social content calendar, coordinating with the creative team, writing editorial copy, planning promotions & contests, etc.\n• Monitor effective benchmarks for measuring the impact of social media programs, and analyze, review, and report on effectiveness of campaigns in an effort to maximize...\n'

In [ ]:
resumes[3]['text']

'Austin Long\nDigital Marketing Manager\na.long@email.com\n(123) 456-7890\nChicago, IL\nLinkedIn\nWORK EXPERIENCE\nHoward Hughes Corporation - Digital Marketing Manager\nJanuary 2018 - current\nChicago, IL\nOwned email planning and execution of email marketing campaigns, resulting in attributable revenue of\n$2.2M\nOptimized email copy and email sequences through A/B testing to improve the average open rate from 8%\nto 14%\nLed customer segmentation efforts to trigger automated tagging and sending of emails based on user\ninteractions with customer platforms\nImproved the average free trial to paid conversion rate by 15% with the use of customized email sequences,\nresulting in $300,000 in new revenue\nIdentified user actions that strongly correlated with user churn, and intervened via email sequences to\nreduce the average monthly churn rate from 7% to 5%\nStripe - Digital Marketing Manager\nApril 2015 - January 2018\nChicago, IL\nDeveloped a comprehensive paid acquisition strategy ac

##**2. Keyword Extraction and Matching:**

**2.1 Using YAKE**

In [ ]:
import yake

def extract_keywords(text : str, numOfKeywords = 100,language = "en",max_ngram_size = 3,deduplication_threshold = 0.9) -> list:
    """
    Extracts keywords from a given text using YAKE.
    """
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    return keywords

In [ ]:
%%time
keywords = extract_keywords(description,60)

CPU times: user 430 ms, sys: 8.01 ms, total: 438 ms
Wall time: 431 ms


In [ ]:
keywords_only = [kw[0] for kw in keywords]
print(keywords_only)

['Digital Marketing Executive', 'developing IIAS’ presence', 'Executive is responsible', 'offered by IIAS', 'IIAS and support', 'Marketing Executive', 'Digital Marketing', 'Job Summary', 'digital marketing campaigns', 'digital platforms', 'responsible to generate', 'generate enquiries', 'support the admission', 'admission services', 'services by developing', 'developing IIAS’', 'IIAS’ presence', 'Digital', 'Summary The Digital', 'Summary', 'Executive', 'IIAS', 'Marketing', 'digital channels', 'campaigns', 'Job Description', 'marketing campaigns', 'marketing team', 'presence across digital', 'design', 'design and development', 'platforms', 'social media', 'social', 'Job', 'team', 'Design and execute', 'responsible', 'generate', 'enquiries', 'offered', 'support', 'admission', 'services', 'developing', 'IIAS’', 'presence', 'social media campaigns', 'track and optimize', 'implement', 'Actively strategize', 'optimize our digital', 'media', 'implement innovative social', 'media campaigns', '

**2.2 Using Tf-idf**

In [ ]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string
import nltk

# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Preprocess text
def preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.text not in stopwords.words('english') and token.text not in string.punctuation and not token.is_stop]
    return ' '.join(tokens)

processed_text = preprocess(description)

# Extract keywords using TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([processed_text])
feature_names = vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.toarray().flatten()

# Get keywords with the highest TF-IDF scores
keywords = {feature_names[i]: tfidf_scores[i] for i in range(len(feature_names))}
sorted_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)

# Print top N keywords
top_n = 100  # Number of top keywords to display
for keyword, score in sorted_keywords[:top_n]:
    print(f"{keyword}: {score}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


ai: 0.6
learning: 0.26666666666666666
machine: 0.2
system: 0.2
computer: 0.13333333333333333
datum: 0.13333333333333333
deep: 0.13333333333333333
develop: 0.13333333333333333
drive: 0.13333333333333333
engineer: 0.13333333333333333
model: 0.13333333333333333
skill: 0.13333333333333333
solution: 0.13333333333333333
team: 0.13333333333333333
ability: 0.06666666666666667
analysis: 0.06666666666666667
architecture: 0.06666666666666667
artificial: 0.06666666666666667
brief: 0.06666666666666667
broad: 0.06666666666666667
building: 0.06666666666666667
business: 0.06666666666666667
candidate: 0.06666666666666667
collaborate: 0.06666666666666667
contribute: 0.06666666666666667
current: 0.06666666666666667
degree: 0.06666666666666667
deploy: 0.06666666666666667
deployment: 0.06666666666666667
development: 0.06666666666666667
dynamic: 0.06666666666666667
engineering: 0.06666666666666667
enhancement: 0.06666666666666667
ensure: 0.06666666666666667
environment: 0.06666666666666667
excellent: 0.0666

**2.3 Matching Resume**

In [ ]:
def match_resumes_to_keywords(resumes, keywords, top_k):
    ranked_resumes = []
    for resume in resumes:
        resume_keywords = extract_keywords(resume['text'])
        score = sum(1 for keyword in keywords if keyword in resume_keywords)
        ranked_resumes.append((resume, score))
    ranked_resumes.sort(key=lambda x: x[1], reverse=True)
    return [resume for resume, score in ranked_resumes[:top_k]]

In [ ]:
matched_resumes = match_resumes_to_keywords(resumes, keywords_only, 2)

In [ ]:
matched_resumes

[{'filename': 'dummy4.pdf',
  'page_count': 1,
  'page_char_count': 2424,
  'page_word_count': 275,
  'page_sentence_count_raw': 3,
  'page_token_count': 606.0,
  'text': 'TANIA SOLIS\nDigital Marketer\nt.solis@email.com\n(123) 456-7890\nBrooklyn, NY\nLinkedIn\nWORK EXPERIENCE\nDigital Marketing Manager\nDesignity\nJanuary 2015 - current\nNew York, NY\nStrategized, developed, and managed paid digital marketing across\nAdWords, Instagram, and Facebook with monthly budget of\n$160,000, resulting in about $645,000 in monthly revenue\nDeveloped robust conditional email campaigns based on customer\ninteractions with the platform that improved retention by 23%\nCreated reporting around paid marketing funnels, and leveraged\nthis to incrementally improve the conversion rate by 210%\nEstablished in-house knowledge of marketing content leveraged by\nthe sales team to improve customer LTV by 28%\nLaunched SEO campaign for high volume and long-tail keywords\nthat generated 11,000 unique visitors 

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def match_resumes_to_keywords(resumes, keywords, top_k):
    model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    model.to("cuda") # requires a GPU installed
    # Create embeddings for the keywords
    keyword_embeddings = model.encode(keywords)

    ranked_resumes = []
    for resume in tqdm(resumes):
        # Create embedding for the resume text
        resume_embedding = model.encode(resume['text'])
        total_score = 0

        for keyword_embedding in keyword_embeddings:
            # Calculate the cosine similarity
            score = cosine_similarity([keyword_embedding], [resume_embedding]).flatten()[0]
            total_score += score

        # Normalize the score
        normalized_score = total_score / len(keywords)

        ranked_resumes.append((resume, normalized_score))

    # Sort the resumes by the normalized cosine similarity score in descending order
    ranked_resumes.sort(key=lambda x: x[1], reverse=True)

    # Print normalized scores and return the top_k resumes
    for resume, score in ranked_resumes:
        print(f"Resume: {resume['text']}\nNormalized Score: {score:.4f}\n")

    return [resume for resume, score in ranked_resumes[:top_k]]


top_k = 2
matched_resumes = match_resumes_to_keywords(resumes, keywords_only, top_k)
for resume in matched_resumes:
    print(resume)


  0%|          | 0/5 [00:00<?, ?it/s]

Resume: TANIA SOLIS
Digital Marketer
t.solis@email.com
(123) 456-7890
Brooklyn, NY
LinkedIn
WORK EXPERIENCE
Digital Marketing Manager
Designity
January 2015 - current
New York, NY
Strategized, developed, and managed paid digital marketing across
AdWords, Instagram, and Facebook with monthly budget of
$160,000, resulting in about $645,000 in monthly revenue
Developed robust conditional email campaigns based on customer
interactions with the platform that improved retention by 23%
Created reporting around paid marketing funnels, and leveraged
this to incrementally improve the conversion rate by 210%
Established in-house knowledge of marketing content leveraged by
the sales team to improve customer LTV by 28%
Launched SEO campaign for high volume and long-tail keywords
that generated 11,000 unique visitors and 160 customers monthly
Digital Marketing Analyst
ShopKeep
September 2012 - January 2015
Washington, DC
Performed rigorous A/B testing, which improved the conversion
rate of marketing

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def match_resumes_to_job_description(resumes, job_description, top_k):
    model = SentenceTransformer('all-MiniLM-L6-v2', device = device)
    model.to("cuda") # requires a GPU installed

    # Create embedding for the job description
    job_description_embedding = model.encode(job_description)

    ranked_resumes = []
    for resume in tqdm(resumes):
        # Create embedding for the resume text
        resume_embedding = model.encode(resume['text'])

        # Calculate the cosine similarity
        score = cosine_similarity([job_description_embedding], [resume_embedding]).flatten()[0]

        ranked_resumes.append((resume, score))

    # Sort the resumes by the cosine similarity score in descending order
    ranked_resumes.sort(key=lambda x: x[1], reverse=True)

    # Print normalized scores and return the top_k resumes
    max_score = ranked_resumes[0][1] if ranked_resumes else 1
    for resume, score in ranked_resumes:
        normalized_score = score / max_score  # Normalize scores
        print(f"Resume: {resume['text']}\nNormalized Score: {normalized_score:.4f}\n")

    return [resume for resume, score in ranked_resumes[:top_k]]

top_k = 2
matched_resumes = match_resumes_to_job_description(resumes, description, top_k)
for resume in matched_resumes:
    print(f"Top Resume: {resume['text']}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


  0%|          | 0/5 [00:00<?, ?it/s]

Resume: TANIA SOLIS
Digital Marketer
t.solis@email.com
(123) 456-7890
Brooklyn, NY
LinkedIn
WORK EXPERIENCE
Digital Marketing Manager
Designity
January 2015 - current
New York, NY
Strategized, developed, and managed paid digital marketing across
AdWords, Instagram, and Facebook with monthly budget of
$160,000, resulting in about $645,000 in monthly revenue
Developed robust conditional email campaigns based on customer
interactions with the platform that improved retention by 23%
Created reporting around paid marketing funnels, and leveraged
this to incrementally improve the conversion rate by 210%
Established in-house knowledge of marketing content leveraged by
the sales team to improve customer LTV by 28%
Launched SEO campaign for high volume and long-tail keywords
that generated 11,000 unique visitors and 160 customers monthly
Digital Marketing Analyst
ShopKeep
September 2012 - January 2015
Washington, DC
Performed rigorous A/B testing, which improved the conversion
rate of marketing

In [ ]:
matched_resumes = match_resumes_to_job_description(resumes, description,2)

In [ ]:
matched_resumes

[{'filename': 'dummy4.pdf',
  'page_count': 1,
  'page_char_count': 2424,
  'page_word_count': 275,
  'page_sentence_count_raw': 3,
  'page_token_count': 606.0,
  'text': 'TANIA SOLIS\nDigital Marketer\nt.solis@email.com\n(123) 456-7890\nBrooklyn, NY\nLinkedIn\nWORK EXPERIENCE\nDigital Marketing Manager\nDesignity\nJanuary 2015 - current\nNew York, NY\nStrategized, developed, and managed paid digital marketing across\nAdWords, Instagram, and Facebook with monthly budget of\n$160,000, resulting in about $645,000 in monthly revenue\nDeveloped robust conditional email campaigns based on customer\ninteractions with the platform that improved retention by 23%\nCreated reporting around paid marketing funnels, and leveraged\nthis to incrementally improve the conversion rate by 210%\nEstablished in-house knowledge of marketing content leveraged by\nthe sales team to improve customer LTV by 28%\nLaunched SEO campaign for high volume and long-tail keywords\nthat generated 11,000 unique visitors 

**3. Summarization:**

In [ ]:
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 15 GB


In [ ]:
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False
    model_id = "google/gemma-7b-it"

GPU memory: 15 | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.


In [ ]:
!git config --global credential.helper 'store --file ~/.my-credentials'

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available

# 1. Create quantization config for smaller model loading.
# For models that require 4-bit quantization (use this if low GPU memory is available)
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")

# 2. Pick a model we'd like to use
model_id = model_id # (we already set this above)
print(f"[INFO] Using model_id: {model_id}")

# 3. Instantiate tokenizer (tokenizer turns text into numbers ready for the model)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

# 4. Instantiate the model
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                                 torch_dtype=torch.float16, # datatype to use, we want float16
                                                 quantization_config=quantization_config if use_quantization_config else None,
                                                 low_cpu_mem_usage=False, # use full memory
                                                 attn_implementation=attn_implementation) # which attention version to use

if not use_quantization_config: # quantization takes care of device setting automatically, so if it's not used, send model to GPU
    llm_model.to("cuda")

[INFO] Using attention implementation: sdpa
[INFO] Using model_id: google/gemma-2b-it


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [ ]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

2506172416

In [ ]:
def get_model_mem_size(model: torch.nn.Module):
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate various model sizes
    model_mem_bytes = mem_params + mem_buffers # in bytes
    model_mem_mb = model_mem_bytes / (1024**2) # in megabytes
    model_mem_gb = model_mem_bytes / (1024**3) # in gigabytes

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(llm_model)

{'model_mem_bytes': 5012354048, 'model_mem_mb': 4780.15, 'model_mem_gb': 4.67}

In [ ]:
def prompt_formatter(context_items) -> str:
    """
    Augments query with text-based context from context_items.

    """
    # Create a base prompt with examples to help the model
    base_prompt = "Summarize the following resume:\n"
    base_prompt += context_items['text']

    # Create prompt template for instruction-tuned model
    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]

    # Apply the chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [ ]:
def summarize_resume(resume):
    text = resume['text']
    prompt = prompt_formatter(text)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = llm_model.generate(**input_ids,
                                temperature=0.7, # lower temperature = more deterministic outputs, higher temperature = more creative outputs
                                do_sample=True,
                                max_new_tokens=512) # how many new tokens to generate from prompt

    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])
    return output_text

In [ ]:
%%time
for resume in matched_resumes:
  query = resume['text']
  prompt = prompt_formatter(resume)
  input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

  # Generate an output of tokens
  outputs = llm_model.generate(**input_ids,
                              temperature=0.7, # lower temperature = more deterministic outputs, higher temperature = more creative outputs
                              do_sample=True,
                              max_new_tokens=512) # how many new tokens to generate from prompt

  # Turn the output tokens into text
  output_text = tokenizer.decode(outputs[0])

  # print(f"Query: {query}")
  print(f"RAG answer:\n{output_text.replace(prompt, '')}")

RAG answer:
<bos>## Summary of Resume:

**TANIA SOLIS** is a seasoned digital marketer with 9+ years of experience in SaaS companies. She excels at developing and executing strategic paid and organic marketing funnels that generate significant revenue for the businesses she works with.

**Key Achievements:**

* **Digital Marketing Manager, Designity (2015-Present):**
    * Increased monthly revenue by $645,000 through paid marketing campaigns.
    * Improved conversion rate by 210% through email campaign optimization.
    * Established in-house knowledge of marketing content, leading to a 28% increase in customer lifetime value.
    * Launched an SEO campaign for high-volume keywords, resulting in 11,000 unique visitors and 160 customers per month.
* **Digital Marketing Analyst, ShopKeep (2012-2015):**
    * Improved on-page SEO performance by 70%, leading to a 24% increase in monthly organic traffic.
    * Created compelling content, including infographics and white papers, that gener

**4. Detailed Analysis and Feedback Generation:**

In [ ]:
!pip install language_tool_python

In [ ]:
import language_tool_python

def analyze_resume(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    feedback = []
    for match in matches:
        feedback.append({
            "error": match.message,
            "suggestion": match.replacements,
            "context": match.context,
        })
    return feedback


In [ ]:
print(analyze_resume(matched_resumes[0]['text']))

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpnfif80wb.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


[{'error': 'Possible spelling mistake found.', 'suggestion': ['Designing', 'Designate', 'Dignity', 'Benignity'], 'context': '...RK EXPERIENCE Digital Marketing Manager Designity January 2015 - current New York, NY Str...'}, {'error': 'In 2018 Google renamed their advertising platform to “Google Ads”.', 'suggestion': ['Google Ads'], 'context': '...d managed paid digital marketing across AdWords, Instagram, and Facebook with monthly b...'}, {'error': 'Possible spelling mistake found.', 'suggestion': ['Shop Keep'], 'context': '...omers monthly Digital Marketing Analyst ShopKeep September 2012 - January 2015 Washingto...'}, {'error': 'If you want to indicate numerical ranges or time ranges, consider using an en dash.', 'suggestion': ['–'], 'context': '...arketing Analyst ShopKeep September 2012 - January 2015 Washington, DC Performed ri...'}, {'error': 'Possible spelling mistake found.', 'suggestion': ['Italy', 'talk', 'stalk', 'Ital', 'ital'], 'context': '...work each week Digital Marketi